# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute_target import ComputeTargetException

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoMLversion'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FF6G68NJ6 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
amlcompute_cluster_name = "automl-compute"

# Verify that cluster does not exist already
try:
    training_cluster = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    training_cluster = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
found = False
key = "**Breast Cancer Classification**"
description_text = "**Breast Cancer Classification Dataset for Capstone Peoject Udacity**"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/vagbharathi/Capstone-project-UdacityMachineLearningEngineer-MicrosoftAzure-Scholarship/main/nd00333-capstone-master/starter_file/breast_cancer_data.csv
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Favorite Color,Favorite Music Genre,Favorite Beverage,Favorite Soft Drink,Gender
count,66,66,66,66,66
unique,3,7,6,4,2
top,Cool,Rock,Doesn't drink,Coca Cola/Pepsi,M
freq,37,19,14,32,33


In [6]:
dataset.take(5).to_pandas_dataframe()

,Favorite Color,Favorite Music Genre,Favorite Beverage,Favorite Soft Drink,Gender
0,Cool,Rock,Vodka,7UP/Sprite,F
1,Neutral,Hip hop,Vodka,Coca Cola/Pepsi,F
2,Warm,Rock,Wine,Coca Cola/Pepsi,F
3,Warm,Folk/Traditional,Whiskey,Fanta,F
4,Cool,Rock,Vodka,Coca Cola/Pepsi,F


In [ ]:
# Split data into train and test sets
train, test = dataset.random_split(percentage=0.75, seed=121)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=training_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Gender",   
                             enable_early_stopping= True,
                             n_cross_validations=5,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoMLversion,AutoML_dc2020ee-9a83-4286-abbe-67cae4d0bb33,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
AutoMLversion,AutoML_dc2020ee-9a83-4286-abbe-67cae4d0bb33,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  High cardinality features were detected in your inputs and handled.
              Learn more about high c

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: AutoMLversion,
Id: AutoML_dc2020ee-9a83-4286-abbe-67cae4d0bb33_53,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  max_features='log2',
                                                                                                  max_leaf_nodes=None,
                           

In [12]:
#TODO: Save the best model
from azureml.core.model import Model
best_run.register_model(model_path='outputs/model.pkl', model_name='breastcancer_bestautoml', model_framework=Model.Framework.SCIKITLEARN)
model = ws.models['breastcancer_bestautoml']

In [13]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-143135', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-143135'), name=genderbest_automlmodel, id=genderbest_automlmodel:2, version=2, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core import Environment
best_run.download_file('outputs/conda_env_v_1_0_0.yml','conda_env.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="conda_env.yml")
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(entry_script='score.py',
                                   environment=myenv)

service_name = 'automl-service'
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True)
aci_service = Model.deploy(workspace=ws,
                           name= service_name,
                           models= [model],
                           inference_config= inference_config,
                           deployment_config=deployment_config)
aci_service.wait_for_deployment(show_output = True)

In [ ]:
print(aci_service.state)

In [ ]:
# Enable AppInsights
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
endpoint = aci_service.scoring_uri
primary, secondary = aci_service.get_keys()
print(primary)
print(endpoint)

In [ ]:
!python endpoint.py

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
aci_service.delete()
print('Service deleted.')